In [2]:
!pip install PyPDF2
!pip install requests
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.5 MB/s eta 0:00:00


In [5]:
import spacy
from spacy import displacy
import re
NER = spacy.load("en_core_web_sm")
import PyPDF2
import requests
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
import pickle
import json
import pandas as pd
import os
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## IOCS

In [7]:
def extract_iocs(rep_text):

    # fixed_text = fix_broken_hashes(rep_text)

    report_text = rep_text.replace("[.]", ".")

    # Regex patterns
    ip_pattern = r'\b(?:\d{1,3}\.){3}\d{1,4}\b'
    # domain_pattern = r'\b(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\.)+[a-z]{2,}\b'
    domain_pattern = r'\b(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\.)+(?:com|org|net|edu|gov|io|co|biz|http|https|[a-z]{2})\b'
    #|[a-z]{2} added

    # email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b'

    # Hash patterns
    md5_pattern = r'\b[a-f0-9\s]{32,36}\b'
    sha1_pattern = r'\b[a-f0-9\s]{40,44}\b'
    sha256_pattern = r'\b[a-f0-9\s]{64,68}\b'
    sha512_pattern = r'\b[a-f0-9\s]{128,132}\b'
    ssdeep_pattern = r'\b\d{1,}:[A-Za-z0-9+/=]{1,}:[A-Za-z0-9+/=]{1,}\b'
    tlsh_pattern = r'\b[A-Fa-f0-9]{70,72}\b'

    iocs = {
        "IP addresses": re.findall(ip_pattern, report_text),
        "Domains": re.findall(domain_pattern, report_text),
        "File hashes": {
            "MD5": re.findall(md5_pattern, report_text),
            "SHA1": re.findall(sha1_pattern, report_text),
            "SHA256": re.findall(sha256_pattern, report_text),
            "SHA512": re.findall(sha512_pattern, report_text),
            "SSDEEP": re.findall(ssdeep_pattern, report_text),
            "TLSH": re.findall(tlsh_pattern, report_text)
        },
    }

    cleaned_sha256_hashes = []

    for each in iocs['File hashes']['SHA256']:
        # Remove spaces and newlines
        cleaned = each.replace(' ', '').replace('\n', '')[:64]

        # Check if the hash length is 32 (valid MD5)
        cleaned_sha256_hashes.append(cleaned)

    # Replace the original list with the cleaned list
    iocs['File hashes']['SHA256'] = cleaned_sha256_hashes

    cleaned_md5_hashes = []

    for each in iocs['File hashes']['MD5']:
        # Remove spaces and newlines
        cleaned = each.replace(' ', '').replace('\n', '')

        # Check if the hash length is 32 (valid MD5)
        if len(cleaned) == 32:
            cleaned_md5_hashes.append(cleaned)

    # Replace the original list with the cleaned list
    iocs['File hashes']['MD5'] = cleaned_md5_hashes

    cleaned_sha1_hashes = []

    for each in iocs['File hashes']['SHA1']:
        # Remove spaces and newlines
        cleaned = each.replace(' ', '').replace('\n', '')

        # Check if the hash length is 32 (valid MD5)
        if len(cleaned) == 40:
            cleaned_sha1_hashes.append(cleaned)

    # Replace the original list with the cleaned list
    iocs['File hashes']['SHA1'] = cleaned_sha1_hashes

    cleaned_sha512_hashes = []

    for each in iocs['File hashes']['SHA512']:
        # Remove spaces and newlines
        cleaned = each.replace(' ', '').replace('\n', '')

        # Check if the hash length is 32 (valid MD5)
        if len(cleaned) == 128:
            cleaned_sha512_hashes.append(cleaned)

    # Replace the original list with the cleaned list
    iocs['File hashes']['SHA512'] = cleaned_sha512_hashes

    # print(iocs['File hashes']['SHA256'])
    return iocs

In [44]:
# Your VirusTotal API key
API_KEY = "dec069ea4b99701acf7f01cee12b574c0528b6706bac073256bbd22117758bad"

# Function to get a file report based on hash
def get_file_report(hash_value):
    # tags_list = []
    url = f"https://www.virustotal.com/api/v3/files/{hash_value}"
    headers = {
        "x-apikey": API_KEY
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            # Print basic details
            # print(f"Hash: {hash_value}")
            # print(f"Malicious Detections: {data['data']['attributes']['last_analysis_stats']['malicious']}")

            if data['data']['attributes']['last_analysis_stats']['malicious'] >0:

              # print('Tags_List: ', tags_list)
              tags = data['data']['attributes'].get('tags', [])

              for each in tags:
                if each in tags_list:
                  pass
                else:
                  tags_list.append(each)

              # if tags:
              #     print(f"Tags: {', '.join(tags)}")
              # else:
              #     print("Tags: None")
        elif response.status_code == 404:
            print()#f"No data found for hash: {hash_value}"
        else:
            print(f"Error {response.status_code}: {response.json()['error']['message']}")
        # return tags_list
    except Exception as e:
        print(f"An error occurred: {e}")

# Function to get IP address report
def get_ip_report(ip_address):
    url = f"https://www.virustotal.com/api/v3/ip_addresses/{ip_address}"
    headers = {
        "x-apikey": API_KEY
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()

            # # General info about the IP
            # print(f"\n--- IP Report: {ip_address} ---")
            # print(f"Reputation: {data['data']['attributes'].get('reputation', 'N/A')}")
            # print(f"Country: {data['data']['attributes']['country']}")

            # Detected URLs
            detected_urls = data['data']['attributes'].get('last_analysis_results', {})
            # print("\nDetected URLs:")
            # for engine, result in detected_urls.items():
            #     if result['category'] == 'malicious':
            #         print(f"- {engine}: {result['result']}")

            # Associated Domains
            domains = data['data']['attributes'].get('last_analysis_results', {}) #ye krna h??

            # print("\nNetwork Details:")
            # print(f"ASN: {data['data']['attributes']['asn']}")
            # print(f"ISP: {data['data']['attributes']['isp']}")
            # print(f"Network: {data['data']['attributes']['network']}")

        # elif response.status_code == 404:
        #     print(f"No data found for IP: {ip_address}")
        # else:
        #     print(f"Error {response.status_code}: {response.json()['error']['message']}")
    except Exception as e:
        print(f"An error occurred: {e}")

def search_domain(domain):
    url = f"https://www.virustotal.com/api/v3/domains/{domain}"
    headers = {
        "x-apikey": API_KEY
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.json())
        return None

## TTPs

### Tactics

In [9]:
mitre_dict= {
    "Tactics": {
        "TA0001": ["initial access", "network perimeter vulnerability"],
        "TA0002": ["execution", "malware execution", "run malicious code"],
        "TA0008": ["lateral movement", "move laterally", "spread within the network"],
        "TA0027 - Initial Access": ["Phishing", "Exploitation", "Credential theft", "Backdoor"],
        "TA0041 - Execution": ["Malware", "Scripting", "Payload", "Execution flow"],
        "TA0028 - Persistence": ["Backdoor", "Registry", "Scheduled tasks", "Startup"],
        "TA0029 - Privilege Escalation": ["Admin rights", "Exploits", "System vulnerabilities", "Elevated access"],
        "TA0030 - Defense Evasion": ["Obfuscation", "Rootkit", "Anti-analysis", "Encryption"],
        "TA0031 - Credential Access": ["Passwords", "Keylogging", "Credential dumping", "Hash extraction"],
        "TA0032 - Discovery": ["Network mapping", "System info", "Enumeration", "Asset identification"],
        "TA0033 - Lateral Movement": ["Remote access", "Credential reuse", "Lateral traversal", "RDP"],
        "TA0035 - Collection": ["Data theft", "Keylogging", "Screenshot capture", "File collection"],
        "TA0037 - Command and Control": ["C2 server", "Beaconing", "Tunneling", "Remote access"],
        "TA0036 - Exfiltration": ["Data transfer", "Compression", "Cloud storage", "Network protocols"],
        "TA0034 - Impact": ["Ransomware", "Data destruction", "Service disruption", "Wiping"],
        "TA0038 - Network Effects": ["Traffic interception", "Packet manipulation", "Network flooding", "MITM (Man-in-the-Middle)"],
        "TA0039 - Remote Service Effects": ["Remote monitoring", "Unauthorized access", "Service manipulation", "Control takeover"]
    },
    "Techniques":{
    "T1548.001 - Setuid and Setgid": ["Setuid and Setgid", "setuid", "setgid", "binary"],
    "T1548.002 - Bypass User Account Control": ["Bypass User Account Control", "UAC", "integrity levels", ""],
    "T1548.003 - Sudo and Sudo Caching": ["Sudo and Sudo Caching", "caching", "sudo", "sudoers"],
    "T1548.004 - Elevated Execution with Prompt": ["Elevated Execution with Prompt", "AuthorizationExecuteWithPrivileges", "root privileges", "API"],
    "T1548.005 - Temporary Elevated Cloud Access": ["Temporary Elevated Cloud Access", "cloud", "just-in-time", "impersonate", "short-term access"],
    "T1548.006 - TCC Manipulation": ["TCC Manipulation", "Transparency", "Consent", "Control", "FDA"],
    "T1134.001 - Token Impersonation/Theft": ["Token Impersonation/Theft",  "DuplicateToken","DuplicateTokenEx","ImpersonateLoggedOnUser"],
    "T1134.002 - 	Create Process with Token": ["Create Process with Token", "CreateProcessWithTokenW", "runas"],
    "T1134.003 - 	Make and Impersonate Token": ["Make and Impersonate Token", "LogonUser", "logon", "session"],
    "T1134.004 - Parent PID Spoofing": ["Parent PID Spoofing", "parent process identifier", "CreateProcess", "svchost", "consent"],
    "T1134.005 - SID-History Injection": ["SID-History Injection", "History Injection",	"SID",	"Active Directory"],
    "T1087.001 - Local Account": ["Local Account", "local system", "follow-on"],
    "T1087.002 - Domain Account": ["Domain Account", "domain", "follow-on"],
    "T1087.003 - Email Account": ["Email Account", "email addresses", "global address lists", "GAL"],
    "T1087.004 - Cloud Account": ["Cloud Account", "SaaS", "cloud service"],
    "T1098.001 - Additional Cloud Credential": ["Additional Cloud Credentials", "credentials"],
    "T1098.002 - Additional Email Delegate Permission": ["Additional Email Delegate Permissions", "additional permission", "email account"],
    "T1098.003 - Additional Cloud Roles": ["Additional Cloud Roles", "additional roles", "IAM policies", "Office 365"],
    "T1098.004 - SSH Authorized Keys": ["SSH Authorized Keys", "SSH", "authorized_keys", "PubkeyAuthentication"],
    "T1098.005 - Device Registration": ["Device Registration", "register ", "multifactor authentication", "device access"],
    "T1098.006 - Additional Container Cluster Roles": ["Additional Container Cluster Roles", "RoleBinding", "ClusterRoleBinding", "Kubernetes"],
    "T1098.007 - 	Additional Local or Domain Groups": ["Additional Local or Domain Groups", "additional local", "domain groups", "ABAC"],
    "T1583.001 - Domains": ["Domains", "acquire domains", "human names"],
    "T1583.002 - DNS Server": ["DNS Server", "Domain Name System", "Command and Control", "Application Layer Protocol"],
    "T1583.003 - Virtual Private Server": ["Virtual Private Server", "Virtual Private Server", "VPS", "virtual machines"],
    "T1583.004 - Server": ["Server", "Drive-by Compromise", "physical servers"],
    "T1583.005 - Botnet": ["Botnet", "booter", "stresser", "DDoS"],
    "T1583.006 - Web Services": ["Web Services", "Exfiltration Over Web Service", "Phishing"],
    "T1583.007 - Serverless": ["Serverless", "serverless infrastructure", "Cloudflare Workers"],
    "T1583.008 - Malvertising": ["Malvertising", "malware", "purchased ads", "search engine results"],
    "T1010 - Application Window Discovery": ["Application Window Discovery", "window listing", "system usage"],
    "T1560.001 - Archive via Utility": ["Archive via Utility", "data compression", "encryption"],
    "T1560.002 - Archive via Library": ["Archive via Library", "data compression", "third-party libraries"],
    "T1560.003 - Archive via Custom Method": ["Archive via Custom Method", "data compression", "custom implementation"],
    "T1123 - Audio Capture": ["Audio Capture", "audio recording", "sensitive conversations"],
    "T1119 - Automated Collection": ["Automated Collection", "internal data", "automated tools"],
    "T1020.001 - Traffic Duplication": ["Traffic Duplication", "exfiltration", "network mirroring"],
    "T1197 - BITS Jobs": ["BITS Jobs", "background tasks", "file transfer"],
    "T1547.001 - Registry Run Keys / Startup Folder": ["Registry Run Keys / Startup Folder", "persistence", "system boot"],
    "T1547.002 - Authentication Package": ["Authentication Package", "DLL execution", "system authentication"],
    "T1547.003 - Time Providers": ["Time Providers", "DLL execution", "time synchronization"],
    "T1595.001 - Scanning IP Blocks": ["Scanning IP Blocks", "reconnaissance", "network traffic"],
    "T1595.002 - Vulnerability Scanning": ["Vulnerability Scanning", "vulnerabilities", "target host"],
    "T1595.003 - Wordlist Scanning": ["Wordlist Scanning", "brute force", "target infrastructure"],
    "T1557.001 - LLMNR/NBT-NS Poisoning and SMB Relay": ["LLMNR/NBT-NS Poisoning and SMB Relay", "name resolution", "network spoofing"],
    "T1557.002 - ARP Cache Poisoning": ["ARP Cache Poisoning", "address resolution", "man-in-the-middle"],
    "T1557.003 - DHCP Spoofing": ["DHCP Spoofing", "network redirection", "malicious server"],
    "T1557.004 - Evil Twin": ["Evil Twin", "Wi-Fi access", "malicious network"],
    "T1071.001 - Web Protocols": ["Web Protocols", "application layer", "network blending"],
    "T1071.002 - File Transfer Protocol": ["File Transfer Protocols", "data transfer", "network blending"],
    "T1071.003 - Mail Protocols": ["Mail Protocols", "email communication", "network blending"],
    "T1071.004 - DNS": ["DNS", "domain communication", "network blending"],
    "T1010 - Application Window Discovery": ["Window titles", "Open applications", "Active processes", "GUI analysis"],
    "T1560 - Archive Collected Data": ["Data compression", "Archiving tools", "Data packaging", "Backup"],
    "T1560.001 - Archive via Utility": ["Zip", "Tar", "Compression tool", "Command-line"],
    "T1560.002 - Archive via Library": ["API", "Archiving library", "Programmatic methods", "Data modules"],
    "T1560.003 - Archive via Custom Method": ["Custom script", "Manual compression", "Proprietary tools", "Custom code"],
    "T1123 - Audio Capture": ["Microphone", "Audio recording", "Voice data", "Surveillance"],
    "T1119 - Automated Collection": ["Script automation", "File sweeps", "Data collection", "Batch processing"],
    "T1020 - Automated Exfiltration": ["File transfer", "Data leakage", "Export scripts", "Network communication"],
    "T1020.001 - Traffic Duplication": ["Packet capture", "Network sniffing", "Traffic mirroring", "Duplicated data"],
    "T1197 - BITS Jobs": ["Background downloads", "File transfer", "BITS service", "Task scheduling"],
    "T1547 - Boot or Logon Autostart Execution": ["Startup scripts", "Boot sequence", "Persistence", "Initialization"],
    "T1547.001 - Registry Run Keys / Startup Folder": ["Registry entries", "Startup programs", "Persistence", "Autostart"],
    "T1547.002 - Authentication Package": ["Credential loading", "Authentication DLL", "Logon process", "Credential theft"],
    "T1547.003 - Time Providers": ["Time synchronization", "NTP", "Service manipulation", "System clock"],
    "T1547.004 - Winlogon Helper DLL": ["Winlogon.exe", "Helper DLL", "Persistence", "Credential theft"],
    "T1547.005 - Security Support Provider": ["SSP DLL", "Authentication mechanisms", "Credential access", "System processes"],
    "T1547.006 - Kernel Modules and Extensions": ["Kernel hooks", "Driver loading", "Low-level manipulation", "Privilege escalation"],
    "T1547.007 - Reopened Applications": ["Application persistence", "Session restoration", "Startup restoration", "Data retention"],
    "T1547.008 - LSASS Driver": ["LSASS.exe", "Credential dumping", "Driver manipulation", "Memory access"],
    "T1547.009 - Shortcut Modification": ["LNK files", "File shortcuts", "Desktop persistence", "File redirection"],
    "T1547.010 - Port Monitors": ["Print spooler", "Port redirection", "Persistence", "Printer monitoring"],
    "T1547.012 - Print Processors": ["Printer driver", "DLL injection", "Persistence", "System components"],
    "T1547.013 - XDG Autostart Entries": ["Linux startup", "Autostart applications", "Desktop files", "Persistence"],
    "T1547.014 - Active Setup": ["Active setup registry", "Windows initialization", "User settings", "Persistence"],
    "T1547.015 - Login Items": ["MacOS", "Startup programs", "Persistence", "User settings"],
    "T1480 - Execution Guardrails": ["Keying mechanisms", "Environment detection", "Execution constraints", "Mitigation"],
    "T1480.001 - Environmental Keying": ["Hardware check", "Environment match", "Execution restriction", "Validation"],
    "T1480.002 - Mutual Exclusion": ["Mutex", "Thread lock", "Execution prevention", "Process synchronization"],
    "T1048 - Exfiltration Over Alternative Protocol": ["Non-standard protocol", "Encrypted exfiltration", "Steganography", "Network bypass"],
    "T1048.001 - Exfiltration Over Symmetric Encrypted Non-C2 Protocol": ["Symmetric encryption", "Data concealment", "Network protocol", "Exfiltration"],
    "T1048.002 - Exfiltration Over Asymmetric Encrypted Non-C2 Protocol": ["Asymmetric encryption", "Data security", "Protocol misuse", "Stealth exfiltration"],
    "T1048.003 - Exfiltration Over Unencrypted Non-C2 Protocol": ["Cleartext data", "Network protocol", "Data leak", "Exfiltration"],
    "T1041 - Exfiltration Over C2 Channel": ["Command-and-control", "Data transfer", "C2 beacon", "Encrypted exfiltration"],
    "T1011 - Exfiltration Over Other Network Medium": ["Unusual medium", "Network layer", "Covert channel", "Data exfiltration"],
    "T1011.001 - Exfiltration Over Bluetooth": ["Bluetooth protocol", "Data transfer", "Wireless communication", "Covert exfiltration"],
    "T1052 - Exfiltration Over Physical Medium": ["Removable media", "Physical device", "Data theft", "Off-network exfiltration"],
    "T1052.001 - Exfiltration Over USB": ["USB drives", "Data copying", "File transfer", "Physical exfiltration"],
    "T1567 - Exfiltration Over Web Service": ["Web API", "Cloud storage", "HTTPS exfiltration", "Web protocol"],
    "T1567.001 - Exfiltration to Code Repository": ["GitHub", "Code storage", "Version control", "Data exfiltration"],
    "T1567.002 - Exfiltration to Cloud Storage": ["Google Drive", "Dropbox", "Cloud-based", "Data upload"],
    "T1567.003 - Exfiltration to Text Storage Sites": ["Pastebin", "Text sharing", "Public exposure", "Covert exfiltration"],
    "T1567.004 - Exfiltration Over Webhook": ["Webhook callback", "API interaction", "Data exfiltration", "Automation"],
    "T1190 - Exploit Public-Facing Application": ["Vulnerability exploitation", "Web app", "Remote access", "Initial access"],
    "T1203 - Exploitation for Client Execution": ["Malware delivery", "Remote code", "User interaction", "Execution"],
    "T1212 - Exploitation for Credential Access": ["Password compromise", "Vulnerability abuse", "System credentials", "Unauthorized access"],
    "T1211 - Exploitation for Defense Evasion": ["Bypass defenses", "Obfuscation", "Anti-security", "Exploitation"],
    "T1068 - Exploitation for Privilege Escalation": ["Privilege abuse", "Vulnerabilities", "Root access", "System control"],
    "T1210 - Exploitation of Remote Services": ["Remote exploitation", "Service vulnerabilities", "Network access", "Unauthorized control"],
    "T1133 - External Remote Services": ["VPN", "RDP", "Third-party access", "Unauthorized access"],
    "T1008 - Fallback Channels": ["Backup communication", "Secondary C2", "Redundancy", "Resilient connection"],
    "T1590 - Gather Victim Network Information": ["Domain properties", "DNS", "Network topology", "IP addresses"],
    "T1590.001 - Domain Properties": ["Registrar", "WHOIS lookup", "DNS records", "Domain age"],
    "T1590.002 - DNS": ["Zone transfers", "DNS queries", "Subdomains", "CNAME records"],
    "T1590.003 - Network Trust Dependencies": ["Trusted IPs", "Peer networks", "Network hierarchy", "Shared resources"],
    "T1590.004 - Network Topology": ["Subnet mapping", "Physical layout", "Network diagram", "Connectivity"],
    "T1590.005 - IP Addresses": ["IP ranges", "Geolocation", "Public addresses", "Private IPs"],
    "T1590.006 - Network Security Appliances": ["Firewalls", "Proxies", "Load balancers", "Intrusion detection systems"],
    "T1591 - Gather Victim Org Information": ["Physical locations", "Business relationships", "Organization chart", "Roles"],
    "T1591.001 - Determine Physical Locations": ["GPS coordinates", "Office locations", "Regional sites", "Headquarters"],
    "T1591.002 - Business Relationships": ["Vendors", "Partners", "Affiliates", "Supply chain"],
    "T1591.003 - Identify Business Tempo": ["Operating hours", "Seasonal cycles", "Work shifts", "Industry trends"],
    "T1591.004 - Identify Roles": ["Leadership", "Key personnel", "Technical contacts", "Support staff"],
    "T1615 - Group Policy Discovery": ["GPO settings", "Active Directory", "Policy objects", "Inheritance rules"],
    "T1200 - Hardware Additions": ["Rogue devices", "Peripheral implants", "USB drops", "Hardware trojans"],
    "T1564 - Hide Artifacts": ["Concealed files", "Hidden users", "Stealth techniques", "Evasion"],
    "T1564.001 - Hidden Files and Directories": ["Hidden folders", "Attributes", "Steganography", "Camouflage"],
    "T1564.002 - Hidden Users": ["Fake accounts", "Unauthorized profiles", "Stealth identities", "Masked sessions"],
    "T1564.003 - Hidden Window": ["Invisible GUI", "Window minimization", "Overlay techniques", "Stealthy execution"],
    "T1564.004 - NTFS File Attributes": ["Alternate data streams", "File metadata", "Attribute modification", "Hidden binaries"],
    "T1564.005 - Hidden File System": ["Virtual drives", "Encrypted volumes", "Stealth partitions", "Hidden mount points"],
   "T1564": ["Stealth techniques", "Obfuscation", "Redirection", "Network masking"],

    "T1574.001": ["DLL hijacking", "Dynamic link library", "LoadLibrary", "File injection"],
    "T1574.002": ["DLL replacement", "Side-loading attacks", "Legitimate application misuse", "Malicious library"],
    "T1574.004": ["Dylib replacement", "macOS hijacking", "Dynamic libraries", "Path manipulation"],
    "T1574.005": ["Insecure installer", "File permission abuse", "Installer hijacking", "Elevated privilege misuse"],
    "T1574.006": ["Dynamic linking", "Shared object abuse", "Linker manipulation", "Library preload"],
    "T1574.007": ["Path variable manipulation", "Command redirection", "Malicious path injection", "System PATH abuse"],
    "T1574.008": ["Search path manipulation", "Executable hijacking", "Application redirection", "Custom search paths"],
    "T1574.009": ["Unquoted path exploitation", "Command injection", "File name manipulation", "Escalation through paths"],
    "T1574.010": ["Service misconfiguration", "File permission abuse", "Service exploitation", "Write-access misuse"],
    "T1574.011": ["Registry misconfiguration", "Registry privilege abuse", "Service modification", "Windows registry exploitation"],
    "T1574.012": ["COR_PROFILER abuse", "Profiling API exploitation", "CLR manipulation", ".NET application abuse"],
    "T1574.013": ["Kernel callback exploitation", "Process table hijacking", "Windows kernel abuse", "Malware persistence"],
    "T1574.014": ["AppDomain hijacking", ".NET injection", "Domain control", "CLR abuse"],

    "T1562.001": ["Security tool bypass", "Tool disabling", "AV evasion", "System utility abuse"],
    "T1562.002": ["Event log manipulation", "Log clearing", "Audit disabling", "Logging service stop"],
    "T1562.003": ["Shell history clearing", "Command log tampering", "Terminal history abuse", "Session obfuscation"],
    "T1562.004": ["Firewall rule modification", "Packet filtering bypass", "Network security evasion", "Host firewall disabling"],
    "T1562.006": ["IoC evasion", "Signature blocking", "Threat indicator suppression", "Detection avoidance"],
    "T1562.007": ["Cloud environment evasion", "Cloud firewall rule changes", "CSP bypass", "Instance firewall disabling"],


    "T1562.008 - Disable or Modify Tools": ["Disable logging", "Modify detection tools", "AV disabling", "Logging service shutdown"],
    "T1562.009 - Disable Windows Event Logging": ["Safe Mode boot", "Limited services mode", "Service restriction", "AV service evasion"],
    "T1562.010 - Downgrade Attack": ["Downgrade attack", "System feature reduction", "Version rollback", "Compatibility exploitation"],
    "T1562.011 - Spoof Security Alerts": ["Security auditing bypass", "Event suppression", "Audit logs manipulation", "Detection evasion"],
    "T1562.012 - Disable or Modify Linux Audit Logs": ["Linux audit evasion", "Linux logging disable", "System monitoring bypass", "Log modification"],

    "T1656 - Impersonation": ["Impersonation", "Identity spoofing", "Credential misuse", "Adversary masquerading"],

    "T1525 - Implant Internal Image": ["Inject images", "Image alteration", "Container compromise", "Virtual environment persistence"],

    "T1070.001 - Indicator Removal": ["Indicator removal", "Artifact deletion", "Trace evasion", "Evidence hiding"],
    "T1070.002 - Clear Windows Event Logs": ["Clear event logs", "Windows log deletion", "Audit log wiping", "Security log erasure"],
    "T1070.003 - Clear Linux or Mac System Logs": ["System logs deletion", "Command history clearing", "Log purge", "Admin trace erasure"],
    "T1070.004 - Clear Command History": ["Command history erasure", "CLI log clearing", "Session wiping", "Terminal trace removal"],
    "T1070.005 - File Deletion": ["File deletion", "Malware cleanup", "Data sanitization", "Forensic trace removal"],
    "T1070.006 - Network Share Connection Removal": ["Network share disconnection", "Share cleanup", "Resource unlinking", "Access removal"],
    "T1070.007 - Timestamp Modification": ["Timestamp modification", "File attribute alteration", "Change log evasion", "Backdating"],
    "T1070.008 - Clear Network Connection History and Configurations": ["Connection log clearing", "Network history erasure", "Device tracking suppression", "Configuration hiding"],
    "T1070.009 - Clear Mailbox Data": ["Mailbox clearing", "Email log wiping", "Communication trace removal", "Message deletion"],
    "T1070.010 - Clear Persistence": ["Persistence removal", "Artifact cleanup", "Hidden backdoor clearing", "Credential erasure"],
    "T1070.011 - Relocate Malware": ["Malware relocation", "File movement", "Executable shifting", "Program re-pathing"],

    "T1202 - Indirect Command Execution": ["Command execution", "Interface bypass", "Command-line utility abuse", "Execution evasion"],
    "T1105 - Ingress Tool Transfer": ["Malware tool transfer", "External tool import", "Adversary transfer", "Payload fetching"],

    "T1548.001 - Inhibit System Recovery": ["System recovery inhibit", "Service delay", "Recovery feature abuse", "Restore function disable"],

    "T1056.001 - Keylogging": ["Keylogging", "Keystroke capture", "Input monitoring", "Credential stealing"],
    "T1056.002 - Clipboard Capture": ["Clipboard monitoring", "Clipboard data theft", "Copy-paste interception", "Credential extraction"],
    "T1534 - Internal Spearphishing": ["Spearphishing emails", "Malicious links", "Compromised accounts", "Email-based attack"],
    "T1570 - Lateral Tool Transfer": ["Tool transfer", "Payload delivery", "File sharing", "Remote upload"],
    "T1654 - Log Enumeration": ["Log files", "Event log analysis", "System monitoring", "System discovery"],
    "T1036 - Masquerading": ["File manipulation", "Metadata spoofing", "Name obfuscation", "Impersonation"],

    "T1036.001 - Invalid Code Signature": ["Code signing", "Invalid certificate", "Tampered signature", "Signature validation bypass"],
    "T1036.002 - Right-to-Left Override": ["RTL override", "Text direction", "File renaming", "Deceptive file names"],
    "T1036.003 - Rename System Utilities": ["Utility renaming", "Process hiding", "Tool manipulation", "Renaming artifacts"],
    "T1036.004 - Masqueraded Task or Service": ["Task name spoofing", "Service manipulation", "Scheduled task masquerade", "Name-based evasion"],
    "T1036.005 - Match Legitimate Name or Location": ["Path spoofing", "File location manipulation", "Legitimate path masquerade", "Resource obfuscation"],
    "T1036.006 - Space after Filename": ["Filename space manipulation", "Trailing space evasion", "File naming trick", "Extension deception"],
    "T1036.007 - Double File Extension": ["File extension trick", "Multiple extensions", "Extension spoofing", "Hidden file type"],
    "T1036.008 - Masquerade File Type": ["File type deception", "Hidden file type", "Format spoofing", "Content masquerade"],
    "T1036.009 - Break Process Trees": ["Process tree evasion", "Parent-child spoofing", "Malware hiding", "Task hiding"],
    "T1036.010 - Masqueraded Account Name": ["Account name spoofing", "Fake user accounts", "Identity masking", "Account-based evasion"],

    "T1556 - Modify Authentication Process": ["Authentication bypass", "Credential spoofing", "Access manipulation", "Multi-factor evasion"],
    "T1556.001 - Domain Controller Authentication": ["Domain controller patching", "Authentication bypass", "Domain-level evasion", "Credential abuse"],
    "T1556.002 - Password Filter DLL": ["Password filter manipulation", "Malicious DLL", "Credential validation bypass", "Authentication process injection"],
    "T1556.003 - Pluggable Authentication Modules": ["PAM module abuse", "Credential spoofing", "Authentication hook", "Linux authentication bypass"],
    "T1556.004 - Network Device Authentication": ["Device credential spoofing", "Network access manipulation", "Authentication evasion", "Protocol misuse"],
    "T1556.005 - Reversible Encryption": ["Encryption bypass", "Credential decryption", "Weak encryption abuse", "Password exposure"],
    "T1556.006 - Multi-Factor Authentication Interception": ["MFA bypass", "Token theft", "Session hijacking", "Authentication manipulation"],
    "T1556.007 - Hypervisor Injection": ["Virtualization manipulation", "VM compromise", "Guest OS attack", "Virtual machine evasion"],
    "T1556.008 - Network Provider DLL": ["DLL injection", "Network stack manipulation", "Custom provider abuse", "Credential theft"],
    "T1556.009 - Conditional Access Policies": ["Access policy manipulation", "Conditional access bypass", "Policy evasion", "Resource restriction avoidance"],

    "T1068 - Exploitation for Privilege Escalation": ["Privilege escalation", "Software vulnerabilities", "Code execution", "Elevated permissions"],
    "T1210 - Exploitation of Remote Services": ["Remote access", "Unauthorized access", "Network services", "Lateral movement"],
    "T1133 - External Remote Services": ["External-facing services", "Remote connections", "VPN", "VNC", "Credential authentication"],
    "T1008 - Fallback Channels": ["Alternate channels", "Backup communication", "Command and control", "Data transfer evasion"],
    "T1083 - File and Directory Discovery": ["File enumeration", "Directory search", "Automated discovery", "File system exploration"],
    "T1222 - File and Directory Permissions Modification": ["Access control evasion", "File permissions", "ACL modification", "Protected files"],
    "T1222.001 - Windows File and Directory Permissions Modification": ["Windows ACLs", "File permissions", "Access control evasion", "Protected files"],
    "T1222.002 - Linux and Mac File and Directory Permissions Modification": ["Linux ACLs", "Mac file permissions", "Access control evasion", "Protected files"],
    "T1657 - Financial Theft": ["Monetary theft", "Extortion", "Fraud", "Cryptocurrency theft"],
    "T1495 - Firmware Corruption": ["Firmware attack", "BIOS corruption", "Device availability", "Hardware failure"],
    "T1187 - Forced Authentication": ["Credential harvesting", "Forced login", "Authentication interception", "Credential gathering"],
    "T1606 - Forge Web Credentials": ["Fake credentials", "Session cookies", "Token forgery", "Authentication bypass"],
    "T1606.001 - Web Cookies": ["Web cookie spoofing", "Session hijacking", "Cookie forgery", "Authentication tokens"],
    "T1606.002 - SAML Tokens": ["SAML token forging", "SSO abuse", "Token signing", "Authentication bypass"],
    "T1592 - Gather Victim Host Information": ["Host reconnaissance", "System profiling", "Host information gathering", "Targeting hosts"],
    "T1592.001 - Hardware": ["Hardware profiling", "Host device analysis", "System components", "Hardware reconnaissance"],
    "T1592.002 - Software": ["Software profiling", "Installed applications", "System software analysis", "Software reconnaissance"],
    "T1046 - Network Service Discovery": ["Service enumeration", "Remote service scanning", "Port and service listing", "Infrastructure probing"],
    "T1135 - Network Share Discovery": ["File share enumeration", "Shared folders", "Network shares", "Lateral movement preparation"],
    "T1040 - Network Sniffing": ["Traffic interception", "Packet capture", "Network monitoring", "Authentication sniffing"],
    "T1095 - Non-Application Layer Protocol": ["Non-application protocols", "OSI layers", "Alternative communication protocols", "Tunneled protocols"],
    "T1571 - Non-Standard Port": ["Custom port usage", "Port obfuscation", "Unusual port selection", "Protocol evasion"],
    "T1027 - Obfuscated Files or Information": ["File obfuscation", "Data encryption", "Information hiding", "Code masking"],
    "T1027.001 - Binary Padding": ["Junk data insertion", "Binary modification", "File size increase", "Binary evasion"],
    "T1027.002 - Software Packing": ["Executable packing", "Code compression", "Signature evasion", "Virtual machine protection"],
    "T1027.003 - Steganography": ["Hidden data", "Digital media encoding", "Information concealment", "Stego techniques"],
    "T1027.004 - Compile After Delivery": ["Payload compilation", "Uncompiled code", "Post-delivery execution", "Code evasion"],
    "T1027.005 - Indicator Removal from Tools": ["Tool modification", "Malware updating", "Signature evasion", "Indicator removal"],
    "T1027.006 - HTML Smuggling": ["HTML payloads", "Embedded files", "Browser-based delivery", "Malware obfuscation"],


    "T1071.005 - Publish/Subscribe Protocols": ["Publish/Subscribe Protocols", "subscribe", "publish", "network blending"],
    "T0800 - Activate Firmware Update Mode": ["activate firmware update mode", "device maintenance"],
    "T0830 - Adversary-in-the-Middle": ["adversary-in-the-middle", "network interception", "traffic manipulation"],
    "T0878 - Alarm Suppression": ["alarm suppression", "disable alerts", "hide activities"],
    "T0802 - 	Automated Collection": ["automated collection", "data gathering", "scripting"],
    "T0895 - Autorun Image": ["autorun image", "removable media", "automatic execution"],
    "T0803 - 	Block Command Message": ["block command message", "prevent instructions", "inhibit response"],
    "T0804 - Block Reporting Message": ["block reporting message", "suppress telemetry", "hide process state"],
    "T0805 - Block Serial COM	": ["block serial COM", "disable serial communication", "interrupt device control"],
    "T0806 - Brute Force I/O": ["brute force I/O", "force device states", "override controls"],
    "T0807 - 	Command-Line Interface": ["change operating mode", "alter device state", "modify functionality"],
    "T0885 - Commonly Used Port": ["Communicate", "Bypass firewalls", "Network detection", "Normal activity"],
    "T0884 - Connection Proxy": ["Connection proxy", "Network traffic", "Intermediary", "Network communications"],
    "T0879 - Damage to Property": ["Damage and destruction", "Property", "Infrastructure", "Equipment"],
    "T0809 - Data Destruction": ["Data destruction", "Malware", "Tools", "Post-intrusion cleanup"],
    "T0811 - Data from Information Repositories": ["Collect data", "Information repositories", "Sensitive data", "Control systems"],
    "T0893 - Data from Local System": ["Collect data", "Local system", "File systems", "Configuration files"],
    "T0812 - Default Credentials": ["Default credentials", "Manufacturer", "Administrative permissions", "Initial configuration"],
    "T0813 - Denial of Control": ["Denial of control", "Process controls", "Loss of communication", "Operator adjustment"],
    "T0814 - Denial of Service": ["Denial-of-Service", "Device functionality", "High volume", "Request handling"],
    "T0815 - Denial of View": ["Denial of view", "Operator oversight", "Communication failure", "Interface recovery"],
    "T0868 - Detect Operating Mode": ["Operating mode", "Controllers", "Key switch", "Vendor and product line"],
    "T0816 - Device Restart/Shutdown": ["Device restart", "Shutdown", "System reboot", "Maintenance functions"],
    "T0817 - Drive-by Compromise": ["Compromised website", "Exploited system", "Regular browsing"],
    "T0871 - Execution through API": ["API", "Communication", "Control software", "Hardware", "Exploit functions"],
    "T0819 - Exploit Public-Facing Application": ["Weaknesses", "Internet-facing software", "Initial access", "Industrial network"],
    "T0820 - Exploitation for Evasion": ["Software vulnerabilities", "Evasion", "Detection", "Disable security", "Circumvent security"],
    "T0890 - Exploitation for Privilege Escalation": ["Vulnerabilities", "Elevate privileges", "Bypass security", "Permission levels"],
    "T0866 - Exploitation of Remote Services": ["Remote services", "Initial access", "Lateral movement", "ICS environment"],
    "T0822 - External Remote Services": ["External remote services", "VPN", "Citrix", "Connect", "Internal network resources"],
    "T0823 - Graphical User Interface": ["GUI", "Access", "Execute tasks", "Visual", "Mouse", "Keyboard"],
    "T0891 - Hardcoded Credentials": ["Hardcoded credentials", "Software", "Firmware", "Unauthorized access"],
    "T0874 - Hooking": ["API function calls", "Processes", "Execution", "Privilege escalation"],
    "T0877 - I/O Image": ["Process values", "Inputs", "Outputs", "PLC", "Scan cycle"],
    "T0872 - Indicator Removal on Host": ["Indicators", "Presence", "System", "Cover up changes", "Evasion", "Detection"],
    "T0883 - Internet Accessible Device": ["Industrial environments", "Systems", "Exposed", "Internet", "Protections"],
   "T0867 - Lateral Tool Transfer": ["Tool transfer", "File transfer", "Staging tools", "Remote execution", "SMB", "Network shares"],
    "T0826 - Loss of Availability": ["Disruption", "Components", "Systems", "Products", "Services"],
    "T0827 - Loss of Control": ["Sustained loss", "Runaway condition", "Commands", "Malicious interference"],
    "T0828 - Loss of Productivity and Revenue": ["Disruption", "Damage", "Availability", "Integrity", "Control system operations", "Revenue loss"],
    "T0837 - Loss of Protection": ["Compromise", "Protective system functions", "Faults", "Abnormal conditions", "Equipment damage", "Process disruptions", "Hazards"],
    "T0880 - Loss of Safety": ["Compromise", "Safety system functions", "Safe operation", "Dangerous conditions", "Process fails safe"],
    "T0829 - Loss of View": ["Sustained loss", "Permanent loss", "ICS equipment", "Manual operation", "Visibility loss", "State of operations"],
    "T0835 - Manipulate I/O Image": ["Manipulate", "I/O image", "PLCs", "Memory manipulation", "Override function", "Testing", "Scan cycle", "Input/Output values"],
    "T0831 - Manipulation of Control": ["Manipulate", "Physical process control", "Set point values", "Tags", "Parameters", "Control devices", "Command processes", "Detection"],
    "T0832 - Manipulation of View": ["Manipulate", "Reported information", "Operators", "Controllers", "Short term", "Sustained", "Process state"],
    "T0849 - Masquerading": ["Disguise", "Malicious application", "File masquerading", "Common programs", "Vendor executables", "Configuration files", "Application naming", "Legitimacy"],
    "T0838 - Modify Alarm Settings": ["Modify", "Alarm settings", "Prevent alerts", "Presence", "Responses"],
    "T0821 - Modify Controller Tasking": ["Modify", "Controller tasking"],
    "T0836 - Modify Parameter": ["Modify parameters", "Control system devices", "Specify options", "Motor processes"],
    "T0889 - Modify Program": ["Modify program", "Add program", "Controller", "Program download", "Online edit", "Program append"],
    "T0839 - Module Firmware": ["Install firmware", "Modular hardware devices", "Separate firmware"],
    "T0801 - Monitor Process State": ["Physical process state", "Trigger actions", "OPC tags", "Historian data", "PLC block", "Network traffic"],
    "T0834 - Native API": ["Native OS API", "Execute functions", "OS services", "Hardware devices", "Memory", "Processes"],
    "T0840 - Network Connection Enumeration": ["Network connection", "Active connections", "Network patterns", "Netstat", "System firmware", "Network sniffing"],
    "T0842 - Network Sniffing": ["Network interface", "Monitor information", "Capture data"],
    "T0861 - Point & Tag Identification": ["Collect point values", "Tag values", "Process environment"],
    "T0843 - Program Download": ["Transfer user program", "Download to controller"],
    "T0845 - Program Upload": ["Upload program", "Gather process information", "Vendor software", "Upload to workstation", "Jump box"],
    "T0873 - Project File Infection": ["Infect project files", "Malicious code", "Engineering software", "Download infected program", "Execution", "Persistence"],
    "T0886 - Remote Services": ["Leverage remote services", "Move between assets", "RDP", "SMB", "SSH"],
    "T0846 - Remote System Discovery": ["Listing systems", "IP address", "Hostname", "Lateral movement", "Discovery"],
    "T0888 - Remote System Information Discovery": ["Detailed information", "Remote systems", "Make/model", "Role", "Device type"],
     "T0847 - Replication Through Removable Media": ["Move onto systems", "Removable media", "Target devices", "Initial access", "Untrusted networks"],
    "T0848 - Rogue Master": ["Setup rogue master", "Control server functions", "Communication", "Outstations", "Disrupt network"],
    "T0851 - Rootkit": ["Deploy rootkits", "Hide malware", "Intercept OS API calls", "Kernel level", "User level"],
    "T0852 - Screen Capture": ["Screen capture", "Workstations", "HMIs", "Process information", "ICS process"],
    "T0853 - Scripting": ["Use scripting languages", "Execute arbitrary code", "Interpreter", "Compiled languages", "Distribute package"],
    "T0881 - Service Stop": ["Stop services", "Disable services", "Render services unavailable", "Incident response", "Data Destruction"],
    "T0865 - Spearphishing Attachment": ["Spearphishing attachment", "Social engineering", "Malware", "Email", "User Execution"],
    "T0856 - Spoof Reporting Message": ["Spoof reporting messages", "Evasion", "Impair process control", "Telemetry data", "Current state"],
    "T0869 - Standard Application Layer Protocol": ["Command and control", "Application layer protocols", "HTTP(S)", "OPC", "RDP", "Telnet", "DNP3", "Modbus", "Disguise actions"],
    "T0862 - Supply Chain Compromise": ["Supply chain compromises", "Control systems environment", "Infected products"],
    "T0894 - System Binary Proxy Execution": ["Proxy execution", "Malicious content", "Signed binaries", "Microsoft-signed files", "Signature validation"],
    "T0857 - System Firmware": ["Update feature", "Firmware patches", "Remote upgrades", "Software update package", "Network updates"],
    "T0882 - Theft of Operational Information": ["Steal operational data", "Production environment", "Design documents", "Future operations"],
    "T0864 - Transient Cyber Asset": ["Transient devices", "ICS networks", "Management functions", "Temporary access", "Third-party contractor access"],
    "T0855 - Unauthorized Command Message": ["Unauthorized commands", "Control system assets", "Unexpected functions", "ICS networks", "Direct instructions"],
    "T0863 - User Execution": ["User interaction", "Malicious code", "Application installation", "Email attachments", "Higher permissions"],
    "T0859 - Valid Accounts": ["Steal credentials", "User or service account", "Default credentials", "Bypass access controls", "Network resources"],
    "T0860 - Wireless Compromise": ["Wireless network access", "Compromise wireless devices", "Unauthorized communications", "Remote distance"],
    "T0887 - Wireless Sniffing": ["Capture RF communication", "Remote control", "Reporting", "Wireless protocols", "Cyber-physical environments"]
}
}


In [10]:
def extract_ttps_from_text(file_path, mitre_dict):
    with open(file_path, 'r') as file:
        text = file.read()

    results = {"Tactics": [], "Techniques": []}

    for category, entries in mitre_dict.items():
        for code, keywords in entries.items():
            for keyword in keywords:
                if re.search(rf"\b{keyword}\b", text, re.IGNORECASE):
                    if category == "Tactics" and code not in results[category]:
                        results[category].append(code)
                    elif category == "Techniques" and code not in results[category]:
                        results[category].append(code)

    return results

## Integrate

In [45]:
folder_path = '/content/data'  # Replace with your folder path
global tags_list
tags_list = []
# Iterate through files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):

        tags_list.clear()
        print(f'Found file: {filename}')

        output = {'IOCs':{'IP Addresses':[], 'Domains':[]},
                  'TTPs':{'Tactics':[], 'Techniques':[]},
                  'Threat Actor(s)':[],
                  'Malware':{'Keywords':[], 'md5':[], 'sha1':[], 'sha256':[], 'sha512':[],'ssdeep':[], 'TLSH':[], 'Tags':[]},
                  'Targeted Entities':[]}


        f = open(file_path,'r')
        g = open(file_path,'r')
        textli = f.readlines()
        raw_text = g.read()
        iocs = extract_iocs(raw_text)

        # print('-------Checking IP Addresses---------')
        # for ip in iocs['IP addresses']:
        #     get_ip_report(ip)

        # print('-------Checking SHA256 hashes---------')
        # Loop through and query each hash
        for hash_value in iocs['File hashes']['SHA256']:
            # print("\n--- Checking Hash ---")
            get_file_report(hash_value)
        # print('-------Checking SHA1 hashes---------')
        for hash_value in iocs['File hashes']['SHA1']:
            # print("\n--- Checking Hash ---")
            get_file_report(hash_value)
        # print('-------Checking SHA512 hashes---------')
        for hash_value in iocs['File hashes']['SHA512']:
            # print("\n--- Checking Hash ---")
            get_file_report(hash_value)
        # print('-------Checking MD5 hashes---------')
        for hash_value in iocs['File hashes']['MD5']:
            # print("\n--- Checking Hash ---")
            get_file_report(hash_value)
        # print('=====> Tags: ',tags_list)


        attack_verbs = ["attack", "target",'track','activity','malicious','malware','inject','infect','deploy','install','phishing', "exploit", "compromise", "affect", "cause", "breach", "hack", "infiltrate", "penetrate", "disrupt", "override", "manipulate", "corrupt", "damage", "sabotage", "bypass", "deceive", "invade", "escalate", "disable", "intercept", "interfere", "subvert"]
        #deploy
        li = []
        threat_actors = []
        ignore =  ['Microsoft', 'Window', 'HTTP', 'Kaspersky', 'XOR', 'DWORD', 'POST', 'PDF']
        malware = []
        for each in textli:
          if any(word in each for word in attack_verbs):
            text1= NER(each)
            for word in text1.ents:
              if word.label_ == 'ORG':
                if word.text not in li and all(x.lower() not in word.text.lower() for x in ignore):
                  li.append(word.text)
                if word.text not in threat_actors and (any(x in word.text for x in ['UNC', 'APT', 'CVE', 'UA','TA'])):# and all(x not in word.text for x in ignore):
                  threat_actors.append(word.text)
                if word.text not in malware and (all(x not in word.text for x in ['UNC', 'APT', 'CVE', 'UA','TA']) and word.text.isupper()):# and all(x not in word.text for x in ignore):
                  malware.append(word.text)

        # print('Keywords: ', end='')
        for every in li:
          if every not in threat_actors:
            output['Malware']['Keywords'].append(every)
            # print(every, end='\t')

        output['Threat Actor(s)'].extend(threat_actors)
        # print("\nThreat actors: ", threat_actors)
        # print("Malware: ", malware)
        # print('-------'*5)
        # print('\n')

        output['Malware']['md5'] = iocs['File hashes']['MD5']
        output['Malware']['sha1'] = iocs['File hashes']['SHA1']
        output['Malware']['sha256'] = iocs['File hashes']['SHA256']
        output['Malware']['sha512'] = iocs['File hashes']['SHA512']
        output['Malware']['ssdeep'] = iocs['File hashes']['SSDEEP']
        output['Malware']['TLSH'] = iocs['File hashes']['TLSH']


        # File path to the input text file
        input_file = file_path  # Replace with your actual file path

        # Extract TTPs and save to JSON
        output_file = f'extracted_ttps_{filename}.json'
        extracted_ttps = extract_ttps_from_text(input_file, mitre_dict)

        with open(output_file, 'w') as json_file:
            json.dump(extracted_ttps, json_file, indent=4)

        # print(f"TTPs extracted and saved to {output_file}.")

        # Open and read the JSON file
        with open(f'/content/extracted_ttps_{filename}.json', 'r') as file:
            data = json.load(file)

        output['TTPs']['Techniques'] = data['Techniques']
        output['TTPs']['Tactics'] = data['Tactics']


        output['Malware']['Tags'] = tags_list
        output['IOCs']['IP Addresses'] = iocs['IP addresses']
        output['IOCs']['Domains'] = iocs['Domains']

        # print(output)


        # Convert Python to JSON
        json_object = json.dumps(output, indent = 4)

        # Print JSON object
        print(json_object)

Found file: Symantec_Graphiron-Russian-Malware-Deployed-Against-Ukraine(02-08-2023).txt

{
    "IOCs": {
        "IP Addresses": [
            "208.67.104.95"
        ],
        "Domains": [
            "agent.py",
            "checkip.amazonaws.com"
        ]
    },
    "TTPs": {
        "Tactics": [
            "TA0027 - Initial Access",
            "TA0041 - Execution",
            "TA0030 - Defense Evasion",
            "TA0031 - Credential Access",
            "TA0032 - Discovery"
        ],
        "Techniques": [
            "T1548.002 - Bypass User Account Control",
            "T1548.006 - TCC Manipulation",
            "T1098.001 - Additional Cloud Credential",
            "T1098.004 - SSH Authorized Keys",
            "T1583.004 - Server",
            "T1583.006 - Web Services",
            "T1583.008 - Malvertising",
            "T1547.001 - Registry Run Keys / Startup Folder",
            "T1547 - Boot or Logon Autostart Execution",
            "T1547.004 - Winlogon Helper